# Carla lane departure prevention

## Import delle librerie

In [151]:
import carla, cv2
import numpy as np
from skimage.measure import LineModelND, ransac
from queue import Queue
from concurrent.futures import ThreadPoolExecutor

import manual_control
from importlib import reload
reload(manual_control)

<module 'manual_control' from 'c:\\Users\\emanu\\Desktop\\universita\\Smart-vehicular-systems\\lane-departure-prevention-CARLA\\manual_control.py'>

## Setup

Connessione al server di Carla e creazione degli elementi principali.

In [152]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

## Dichiarazione funzioni base

In [153]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    spectator_transform = carla.Transform(back_location, transform.rotation)
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.mercedes.coupe_2020', rotation=None):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    if rotation:
        spawn_point.rotation = rotation
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), fov=90.0, width=800, height=600, sensor_tick=0.0):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera_bp.set_attribute('fov', str(fov))
    camera_bp.set_attribute('sensor_tick', str(sensor_tick))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def remove_all(world: carla.World):
    '''
    Remove all actors and sensors from the world.

    Args:
        world: the world to remove actors and sensors from
    '''
    for a in world.get_actors().filter('vehicle.*'):
        a.destroy()
    for a in world.get_actors().filter('sensor.*'):
        a.destroy()

def proportional_spaced_array(pts: list, min_value: float, max_value: float):
    '''
    Generate an array of values that are proportional to the input array in increasing or decreasing.

    Args:
        pts: the input array
        min_value: the minimum value of the output array
        max_value: the maximum value of the output array

    Returns:
        a list of values that are proportional to the input array in increasing or decreasing
    '''
    delta_dist = max_value - min_value
    pts_max = pts[0]
    pts_min = pts[len(pts) - 1]
    delta_pts = pts_max - pts_min

    buffer = []
    for i in range(0, len(pts)):
        buffer.append(delta_dist * (pts_max - pts[i]) / delta_pts + min_value)

    return buffer

## Warp immagine

*ImageWarp* è una classe che fornisce i metodi per deformare un'immagine e per riportarla come originale.

Metodi:

- **img_warp**: deforma o riporta allo stato originale l'immagine.

- **pts_unwarp**: trasforma i punti individuati sull'immagine deformata alla forma che si adatta all'immagine originale.

In [154]:
class ImageWarp():

    def __init__(self, img_height=240, img_width=320, offset=150, src=[[50, 240], [200, 240], [0, 0], [320, 0]], dst=[[135, 240], [150, 240], [0, 0], [320, 0]]):
        '''
        Initialize the ImageWarp object.

        Args:
            img_height: the height of the image
            img_width: the width of the image
            offset: the height offset of the image
            src: the source points of the image
            dst: the destination points of the image
        '''
        self.img_height = img_height
        self.img_width = img_width
        self.warp_offset = offset
        self.src = np.float32(src)
        self.dst = np.float32(dst)
        self.warp_mat = cv2.getPerspectiveTransform(self.src, self.dst)
        self.warp_mat_inv = cv2.getPerspectiveTransform(self.dst,self.src)

    def img_warp(self, img, inv=False, offset=False):
        '''
        Warps an image based on the input parameters

        Args:
            img ([type]): RGB / Gray image
            inv: invers transformation. Defaults to False.
            offset: use offset for warping the image. Defaults to False.

        Returns:
            [type]: warped image
        '''
        ret = []
        temp_img = None

        if offset == True:
            temp_img = img[self.warp_offset:self.warp_offset+self.img_height, 0:self.img_width]
        else:
            temp_img = img

        if inv == False:
            ret = cv2.warpPerspective(temp_img, self.warp_mat, (self.img_width, self.img_height))
        else:
            ret = cv2.warpPerspective(temp_img, self.warp_mat_inv, (self.img_width, self.img_height))
        return ret

    def pts_unwarp(self, pts):
        '''
        Backprojects points from warped image to un-warped image

        Args:
            pts: points to backproject

        Returns:
            the backprojected points
        '''
        return cv2.perspectiveTransform(pts, self.warp_mat_inv)

## Detector dei punti appartenenti alle linee stradali

*Detector* è una classe che fornisce i metodi per individuare i punti appartenenti alle linee stradali che definiscono la carreggiata.

Metodi:

- **img_filter**: rileva i bordi nell'immagine (con *Canny* vengono rilevati tutti i bordi, mentre con *Sobel* solo quelli verticali), li dilata per migliorarne la visibilità, applica una soglia per binarizzare l'immagine e combina i risultati per ottenere un'immagine finale con bordi più definiti e continui.

- **get_lane_detections**: scansiona verticalmente l'immagine per rilevare i bordi delle corsie, calcola l'istogramma delle rilevazioni, trova i picchi, applica un offset per centrare la finestra di rilevamento, memorizza le coordinate dei picchi e filtra i punti anomali utilizzando RANSAC.

- **draw_detections**: disegna i punti rilevati appartenenti alle linee stradali.

In [155]:
class Detector():

    def __init__(self, scan_range={'start': 0, 'stop': 240, 'steps': 20}, scan_window={'height': 15, 'max_adjust': 10}):
        '''
        Initialize the Detector object.

        Args:
            scan_range: the vertical range of the scan
            scan_window: the window of the scan for each step
        '''
        self.scan_range = scan_range
        self.scan_window = scan_window
        self.model = LineModelND()

    def img_filter(self, img):
        '''
        Filters an RGB image to get the lane boundaries.

        Args:
            img ([type]): RGB image

        Returns:
            [type]: RGB image
        '''
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cimg = cv2.Canny(img, 50, 250)
        cimg = cv2.morphologyEx(cimg, cv2.MORPH_DILATE, (1, 1), iterations=5)

        simg = cv2.Sobel(img, cv2.CV_8U, 1, 0, ksize=1)
        simg = cv2.morphologyEx(simg, cv2.MORPH_DILATE, (1, 1), iterations=5)
        _, simg= cv2.threshold(simg, 50, 255, cv2.THRESH_OTSU)
        simg = cv2.morphologyEx(simg, cv2.MORPH_CLOSE, (3,3))

        img = cv2.bitwise_and(cimg, cimg, mask=simg)

        return img

    def get_lane_detections(self, img, start={'x': 105, 'y': 230}, stop={'x': 135, 'y': 230}, use_RANSAC=True):
        '''
        Parses the input image, with virtual sensors, detects the peaks and save the points.

        Args:
            img ([type]): 1 channel gray image
            start: detection area start. Defaults to {'x': 105, 'y': 230}.
            stop: detection area start. Defaults to {'x': 135, 'y': 230}.
            use_RANSAC: Use RANSAC. Defaults to True.

        Returns:
            [type]: detections coordinates
        '''
        adjust = 0
        minx = min(start['x'], stop['x'])
        maxx = max(start['x'], stop['x']) + adjust
        detections = []
        for i in range (self.scan_range['start'], self.scan_range['stop'], self.scan_range['steps']):
            # detections y coordinate
            y = start['y'] - i

            # get detections from a line
            det_line = img[y:y + self.scan_window['height'], minx:maxx]

            # scan an image segment, sum detection
            hist = np.sum(det_line, axis=0)

            # get peak location
            peak = np.argmax(hist)

            # define threshold = average, find peaks
            if hist[peak] > np.average(hist):

                x1 = minx + peak
                y1 = y
                det_mid_x = minx + len(hist) // 2

                adjust = x1 - det_mid_x

                # apply adjust only if in defined range
                if np.abs(adjust) >= self.scan_window['max_adjust']:
                    sign = np.sign(adjust)
                    adjust = sign * self.scan_window['max_adjust']

                minx += adjust
                maxx += adjust

                detections.append([x1, y1])

        if use_RANSAC == True:
            _, inliers = self.filter_outliers(detections)
            if inliers is not None:
                detections = np.array(detections)[inliers]

        return detections

    def filter_outliers(self, data):
        '''
        Apply RUNSAC.

        Args:
            data ([type]): data points

        Returns:
            [type]: filtered list
        '''
        model_robust = None
        inliers = None
        data = np.array(data)
        self.model.estimate(data)

        try:
            model_robust, inliers = ransac(data, LineModelND, min_samples=2, residual_threshold=1, max_trials=200)
        except:
            pass

        return model_robust, inliers

    def draw_detections(self, img, data):
        '''
        Visualize detections.

        Args:
            img ([type]): RGB image
            data ([type]): points detected

        Returns:
            [type]: RGB image with detections
        '''
        limg = img.copy()
        for v in data:
            cv2.circle(limg, (v[0], v[1]), 2, [255], -1)

        return limg


## Interpolatore dei punti individuati

*Interpolator* è una classe che permette di ottenere le linee che meglio approssimano quelle della strada, partendo dai punti rilevati.

Metodi:

- **interpolate**: esegue l'interpolazione polinomiale sui punti di una corsia rilevata, seleziona il miglior grado del polinomio se richiesto, calcola i valori interpolati e restituisce le coordinate interpolate.

- **echidistant_lane**: genera l'altra linea della corsia data la linea identificata.

In [156]:
class Interpolator():

    def __init__(self, max_poly_degree=3):
        '''
        Initialize the Interpolator object.

        Args:
            max_poly_degree: the maximum polynomial degree
        '''
        self.max_poly_degree = max_poly_degree

    def interpolate(self, pts=dict(), ip_params={'start': 0, 'stop': 240, 'steps': 20}, key='mid', equ_selector=False):
        '''
        Takes detected points, find the corresponding polynom that fits the data.

        Args:
            pts ([type], optional): detected points. Defaults to dict().
            ip_params (dict, optional): interpolation info. Defaults to {'start': 0, 'stop': 240, 'steps': 20}.
            key (str, optional): Name of the line. Defaults to 'mid'.
            equ_selector (bool, optional): search the best fitting equation (line / curve, etc.). Defaults to False.

        Returns:
            [type]: interpolated points
        '''
        data = np.array(pts[0][key])

        x_coord = data[:,0]
        y_coord = data[:,1]

        min_mse_pos = self.max_poly_degree

        # polynomial degree selector
        if equ_selector == True:
            # find the best fit
            best_poly = []
            best_fit = []
            for i in range(0, self.max_poly_degree):
                pfit = np.polyfit(y_coord, x_coord,i)
                polynom = np.poly1d(pfit)
                test_y = polynom(x_coord)
                difference = y_coord - test_y
                st_d = np.std(difference)
                best_poly.append(st_d)
                best_fit.append((pfit, polynom))

            # select best polynom
            min_mse_pos = np.argmin(np.array(best_poly))

        # order start from 1, position from 0
        pfit = np.polyfit(y_coord, x_coord, min_mse_pos)
        polynom = np.poly1d(pfit)

        y_ipp = np.float32(np.linspace(ip_params['start'],ip_params['stop'],ip_params['steps']))
        x_ipp = polynom(y_ipp)

        ply_coords = np.column_stack((x_ipp,y_ipp))
        return {key:ply_coords}

    def echidistant_lane(self, warper: ImageWarp, lane_pts, init_dist=115, final_dist=250, lane_side=1):
        '''
        Create echidistant lane based on the input lane points, considering the deformation generated by the prospective of the camera.

        Args:
            warper: ImageWarp object
            lane_pts: points belonging to the lane detected
            init_dist: initial distance (width of the roadway at the most distant point from the camera)
            final_dist: final distance (width of the roadway at the closest point to the camera)
            lane_side: if it is equal to 1, the lane detected is the right one, if it is equal to -1, the lane detected is the left one

        Returns:
            echidistant generated lane points
        '''
        buffer = []
        dist = init_dist
        dist = proportional_spaced_array(lane_pts[0][:, 1], init_dist, final_dist)
        # create the lane points echidistant to the passed points
        for i in range(0, len(lane_pts[0])):
            x = lane_pts[0][i][0]
            y = lane_pts[0][i][1]

            nx = x
            nx = nx - dist[i] * lane_side

            norm_pts = np.float32(np.column_stack((nx, y)))
            buffer.append(norm_pts)

        buffer = np.array(buffer, dtype=np.float32)
        buffer = cv2.perspectiveTransform(buffer, warper.warp_mat)

        return np.array(buffer, dtype=np.float32)

## Settaggi per il processing

- *src* e *dst* rappresentano l'associazione dei punti dell'immagine di partenza con quelli dell'immagine deformata (es: il punto con coordinate (70, 240) corrisponderà al punto (155, 270) nell'immagine trasformata, in modo analogo gli altri punti)
- *scan_range* indica il range verticale nel quale verranno valutati i punti delle linee e il salto da fare per ogni valutazione
- *scan_window* rappresenta la finestra nella quale viene cercato un punto appartenente ad una linea
- *offset* indica l'offset verticale
- *image_width* contiene la larghezza dell'immagine
- *lanes* rappresenta le aree nelle quali vengono valutate le linee della carreggiata

In [157]:

src = [[70, 240], [430, 240], [0, 0], [500, 0]]
dst = [[155, 270], [165, 270], [0, 0], [320, 0]]

scan_range = {'start': 0, 'stop': 240, 'steps': 10}
scan_window = {'height': 8, 'max_adjust': 8}
offset = 150
image_width = 500

lanes = [
    {'label': 'mid', 'detections': {'start': {'x': 120, 'y': 230}, 'stop': {'x': 160, 'y': 230}}},
    {'label': 'right', 'detections': {'start': {'x': 160, 'y': 230}, 'stop': {'x': 200, 'y': 230}}}
]

## Istanziamento degli oggetti per il processing dell'immagine

In [158]:
warper = ImageWarp(img_width=image_width, offset=offset, src=src, dst=dst)
detector = Detector(scan_range=scan_range, scan_window=scan_window)
interpolator = Interpolator(max_poly_degree=2)

## Funzione di processing dell'immagine

Questa funzione rileva le linee di corsia in un'immagine, le interpola per ottenere curve lisce, le trasforma alla prospettiva originale e salva i risultati in una coda, per la visualizzazione.

La linea rilevata viene mostrata in **blu**, mentre la linea generata a partire da quella rilevata viene disegnata in **verde**.

In [159]:
def process_image(image, res):
    '''
    Process the input image, detect lanes, interpolate and draw the echidistant lane.

    Args:
        image: the input image
        res: the queue to put the result
    '''
    try:
        img = image.copy()
        f_img = detector.img_filter(img)
        f_w_img = warper.img_warp(f_img, offset=True)

        color = [1, 1, 1]

        # img_detected_points = f_w_img.copy()
        debug = f_w_img.copy()

        # detect lanes points
        detected_points = {}
        for i, lane in enumerate(lanes):
            detected_points[lane['label']] = detector.get_lane_detections(f_w_img, start=lane['detections']['start'], stop=lane['detections']['stop'], use_RANSAC=True)

            # draw detection area
            start_x = lane['detections']['start']['x']
            start_y = lane['detections']['start']['y']
            stop_x = lane['detections']['stop']['x']
            stop_y = lane['detections']['stop']['y']
            cv2.rectangle(debug, (start_x, start_y), (stop_x, stop_y), (255, 255, 255), 2)

        # select the best lane (the one with the most detected points)
        lane = None
        if detected_points['mid'].shape[0] > detected_points['right'].shape[0]:
            lane = lanes[0]
        else:
            lane = lanes[1]

        # img_detected_points = detector.draw_detections(img_detected_points, detected_points[lane['label']])

        # interpolate the lane points
        interpolated_points = interpolator.interpolate([detected_points], key=lane['label'], equ_selector=False)

        pts = np.array([interpolated_points[lane['label']]])
        cv2.polylines(debug, [np.int32(pts)], False, [155], 2)

        # unwarp the lane points
        unwarped_pts = np.int32(warper.pts_unwarp(pts))
        unwarped_pts_offset = np.add(unwarped_pts, [0, offset])

        color[0] = 255
        cv2.polylines(img, [unwarped_pts_offset], False, color, 2)

        # estimate the echidistant lane
        ed_pts = np.float32(interpolator.echidistant_lane(warper=warper, lane_pts=unwarped_pts, lane_side=1 if lane['label'] == 'right' else -1))
        cv2.polylines(debug, [np.int32(ed_pts)], False, [155], 2)

        # unwarp the echidistant lane points
        ed_unwarped_pts = np.int32(warper.pts_unwarp(ed_pts))
        ed_unwarped_pts_offset = np.add(ed_unwarped_pts, [0, offset])

        color[0] = 1
        color[1] = 255
        cv2.polylines(img, [ed_unwarped_pts_offset], False, color, 2)

        # draw the separation for the lanes
        middle = lanes[0]['detections']['stop']['x']
        cv2.rectangle(debug, (middle, lane['detections']['start']['y'] - 10), (middle, lane['detections']['start']['y'] + 10), (255, 255, 255), 2)
        concat_img = cv2.hconcat([img, cv2.cvtColor(debug[:, :dst[3][0]], cv2.COLOR_GRAY2RGB)])

        res.put(concat_img)

    except Exception as e:
        print(e)
        start_x = lane['detections']['start']['x']
        start_y = lane['detections']['start']['y']
        stop_x = lane['detections']['stop']['x']
        stop_y = lane['detections']['stop']['y']
        cv2.rectangle(f_w_img, (start_x, start_y), (stop_x, stop_y), (255, 255, 255), 2)
        concat_img = cv2.hconcat([image, cv2.cvtColor(f_w_img[:, :dst[3][0]], cv2.COLOR_GRAY2RGB)])

        res.put(concat_img)

## Definizione valori di dimensione

- *camera_width* e *camera_height* sono le dimensioni dell'immagine catturata dalla camera
- *crop_width*, *crop_height* e *height_adjust* sono i valori della porzione di immagine da analizzare

In [160]:
camera_width = 800
camera_height = 600

crop_width = 500
crop_height = 240
height_adjust = 50

## Esecuzione del programma

In [161]:
remove_all(world)

processed_output = Queue()

# setup the simulation environment
game_loop = manual_control.setup()

# get the vehicle and attach the camera
vehicle = world.get_actors().filter('vehicle.*')[0]
front_camera = spawn_camera(attach_to=vehicle, transform=carla.Transform(
        carla.Location(x=0.3, y=0.0, z=1.5), # posizione dello specchietto retrovisore
        carla.Rotation(pitch=-10.0)
    ),
    sensor_tick=0.1,
    width=camera_width, height=camera_height
)

cv2.namedWindow('Processed', cv2.WINDOW_NORMAL)

# callback for the camera
def camera_callback(image):
    '''
    Callback for the camera.

    Args:
        image: the image captured by the camera
    '''
    try:
        video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
        video_output = video_output[:, :, :3]

        # Crop the image (zoom)
        start_x = (video_output.shape[1] - crop_width) // 2
        start_y = (video_output.shape[0] - crop_height) // 2 - height_adjust
        cropped_img = video_output[start_y:start_y + crop_height, start_x:start_x + crop_width]

        # process the image in a separate thread
        with ThreadPoolExecutor() as executor:
            executor.submit(lambda: process_image(cropped_img, processed_output))

    except Exception as e:
        print(e.with_traceback())

# attach the callback to the camera
front_camera.listen(lambda image: camera_callback(image))

# start the game loop
try:
    game_loop.start(processed_output, autopilot=True)
except KeyboardInterrupt:
    cv2.destroyAllWindows()
finally:
    cv2.destroyAllWindows()

INFO: listening to server 127.0.0.1:2000



Welcome to CARLA manual control.

Use ARROWS or WASD keys for control.

    W            : throttle
    S            : brake
    A/D          : steer left/right
    Q            : toggle reverse
    Space        : hand-brake

    L            : toggle next light type
    Z/X          : toggle right/left blinker

    TAB          : change sensor position
    ` or N       : next sensor
    [1-9]        : change to sensor [1-9]
    C            : change weather (Shift+C reverse)

    F1           : toggle HUD
    ESC          : quit

